In [0]:
from pyspark.sql.functions import col, when, struct, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType

In [0]:
structureData = [
    (("James","","Smith"),"36636","M",[(3100,6000),(3100,6000)]),
    (("Michael","Rose",""),"40288","M",[(3100,6000),(3100,6000)]),
    (("Robert","","Williams"),"42114","M",[(1400,8000),(3100,6000)]),
    (("Maria","Anne","Jones"),"39192","F",[(5500,8000),(3100,6000)]),
    (("Jen","Mary","Brown"),"","F",[(-1,-1),(3100,6000)])
  ]

schema = StructType([
    StructField("name",StructType([
        StructField("first_name",StringType(),True),
        StructField("middle_name",StringType(),True),
        StructField("last_name",StringType(),True)
    ]),True),
    StructField("id",StringType(),True),
    StructField("gender",StringType(),True),
    StructField("salary",ArrayType(StructType([
        StructField("base_sal",StringType(),True),
        StructField("new_sal",StringType(),True)
    ]),True),True)
])

In [0]:
df = spark.createDataFrame(structureData,schema)

In [0]:
df.show(truncate=False)

+--------------------+-----+------+----------------------------+
|name                |id   |gender|salary                      |
+--------------------+-----+------+----------------------------+
|{James, , Smith}    |36636|M     |[{3100, 6000}, {3100, 6000}]|
|{Michael, Rose, }   |40288|M     |[{3100, 6000}, {3100, 6000}]|
|{Robert, , Williams}|42114|M     |[{1400, 8000}, {3100, 6000}]|
|{Maria, Anne, Jones}|39192|F     |[{5500, 8000}, {3100, 6000}]|
|{Jen, Mary, Brown}  |     |F     |[{-1, -1}, {3100, 6000}]    |
+--------------------+-----+------+----------------------------+



In [0]:
from pyspark.sql.functions import col
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- base_sal: string (nullable = true)
 |    |    |-- new_sal: string (nullable = true)



In [0]:
df.withColumn('salaries',explode(col('salary'))).select(col('name.first_name'),col('salaries.base_sal')).show()

+----------+--------+
|first_name|base_sal|
+----------+--------+
|     James|    3100|
|     James|    3100|
|   Michael|    3100|
|   Michael|    3100|
|    Robert|    1400|
|    Robert|    3100|
|     Maria|    5500|
|     Maria|    3100|
|       Jen|      -1|
|       Jen|    3100|
+----------+--------+



In [0]:
df.rdd.getNumPartitions()

Out[5]: 8

In [0]:
df.schema.json()

Out[7]: '{"fields":[{"metadata":{},"name":"name","nullable":true,"type":{"fields":[{"metadata":{},"name":"first_name","nullable":true,"type":"string"},{"metadata":{},"name":"middle_name","nullable":true,"type":"string"},{"metadata":{},"name":"last_name","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"gender","nullable":true,"type":"string"},{"metadata":{},"name":"salary","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"base_sal","nullable":true,"type":"string"},{"metadata":{},"name":"new_sal","nullable":true,"type":"string"}],"type":"struct"},"type":"array"}}],"type":"struct"}'

In [0]:
print('hello')

hello


In [0]:
from pyspark.sql import Row, Column

In [0]:
new_row = Row('raj',27)
new_row

Out[13]: <Row('raj', 27)>

In [0]:
data = [Row(name="James,,Smith",lang=["Java","Scala","C++"],state="CA"), 
    Row(name="Michael,Rose,",lang=["Spark","Java","C++"],state="NJ"),
    Row(name="Robert,,Williams",lang=["CSharp","VB"],state="NV")]

df, rdd = spark.createDataFrame(data), spark.sparkContext.parallelize(data)
df.printSchema(), rdd.collect()

root
 |-- name: string (nullable = true)
 |-- lang: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)

Out[16]: (None,
 [Row(name='James,,Smith', lang=['Java', 'Scala', 'C++'], state='CA'),
  Row(name='Michael,Rose,', lang=['Spark', 'Java', 'C++'], state='NJ'),
  Row(name='Robert,,Williams', lang=['CSharp', 'VB'], state='NV')])

In [0]:
for row in rdd.collect():
    print(row.name + "," +str(row.lang))

James,,Smith,['Java', 'Scala', 'C++']
Michael,Rose,,['Spark', 'Java', 'C++']
Robert,,Williams,['CSharp', 'VB']


In [0]:
# to print spark & application ID
spark.sparkContext.appName, spark.sparkContext.applicationId

Out[22]: ('Databricks Shell', 'local-1655047964783')

In [0]:
## column class

data=[("James",23),("Ann",40)]
df=spark.createDataFrame(data).toDF("name.fname","gender")
df.printSchema()
#root
# |-- name.fname: string (nullable = true)
# |-- gender: long (nullable = true)

# Using DataFrame object (df)
df.select(df.gender).show()
df.select(df["gender"]).show()
#Accessing column name with dot (with backticks)
df.select(df["`name.fname`"]).show()

#Using SQL col() function
from pyspark.sql.functions import col
df.select(col("gender")).show()
#Accessing column name with dot (with backticks)
df.select(col("`name.fname`")).show()


root
 |-- name.fname: string (nullable = true)
 |-- gender: long (nullable = true)

+------+
|gender|
+------+
|    23|
|    40|
+------+

+------+
|gender|
+------+
|    23|
|    40|
+------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
+----------+

+------+
|gender|
+------+
|    23|
|    40|
+------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
+----------+



In [0]:
df.printSchema()

root
 |-- name.fname: string (nullable = true)
 |-- gender: long (nullable = true)



In [0]:
df.select('`name.fname`').show()
df.select(df.gender).show()
df.select(df['`name.fname`'],df['gender']).show()
df.select(col('`name.fname`'),col('gender').alias('sex')).show()

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
+----------+

+------+
|gender|
+------+
|    23|
|    40|
+------+

+----------+------+
|name.fname|gender|
+----------+------+
|     James|    23|
|       Ann|    40|
+----------+------+

+----------+---+
|name.fname|sex|
+----------+---+
|     James| 23|
|       Ann| 40|
+----------+---+



In [0]:
from pyspark.sql import Row
data=[Row(name="James",prop=Row(hair="black",eye="blue")),
      Row(name="Ann",prop=Row(hair="grey",eye="black"))]
df=spark.createDataFrame(data)
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- prop: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)



In [0]:
df.select('*').show()

+-----+-------------+
| name|         prop|
+-----+-------------+
|James|{black, blue}|
|  Ann|{grey, black}|
+-----+-------------+



In [0]:
df.select('name',col('name').alias('name_again'), df['prop.hair'].alias('prop_hair'), df.prop.eye.alias('eye_1st'), 'prop.*').show()

+-----+----------+---------+-------+-----+-----+
| name|name_again|prop_hair|eye_1st| hair|  eye|
+-----+----------+---------+-------+-----+-----+
|James|     James|    black|   blue|black| blue|
|  Ann|       Ann|     grey|  black| grey|black|
+-----+----------+---------+-------+-----+-----+



In [0]:
#df col functions
data=[("James","Bond","100",None),
      ("Ann","Varsa","200",'F'),
      ("Tom Cruise","XXX","400",''),
      ("Tom Brand",None,"400",'M')] 
columns=["fname","lname","id","gender"]
df=spark.createDataFrame(data,columns)


In [0]:
df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)



In [0]:
#alias
from pyspark.sql.functions import expr
df.select(df.fname.alias("first_name"), \
          df.lname.alias("last_name")
   ).show()

#Another example
df.select(expr(" fname ||','|| lname").alias("fullName") \
   ).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|     James|     Bond|
|       Ann|    Varsa|
|Tom Cruise|      XXX|
| Tom Brand|     null|
+----------+---------+

+--------------+
|      fullName|
+--------------+
|    James,Bond|
|     Ann,Varsa|
|Tom Cruise,XXX|
|          null|
+--------------+



In [0]:
#asc, desc to sort ascending and descending order repsectively.
df.sort(df.fname.asc()).show()
df.sort(df.fname.desc()).show()
#df.fname.desc()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|     James| Bond|100|  null|
| Tom Brand| null|400|     M|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
+----------+-----+---+------+



In [0]:
#cast
df.select(df.fname,df.id.cast("int")).printSchema()

root
 |-- fname: string (nullable = true)
 |-- id: integer (nullable = true)



In [0]:
df.count()

Out[73]: 4

In [0]:
#between
df.filter(df.id.between(100,300)).show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
|  Ann|Varsa|200|     F|
+-----+-----+---+------+



In [0]:
df.id.between(100,300)

Out[77]: Column<'((id >= 100) AND (id <= 300))'>

In [0]:
#contains
df.filter(df.fname.contains("Cruise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [0]:
df.fname.contains("Cruise")

Out[79]: Column<'contains(fname, Cruise)'>

In [0]:
#startswith, endswith()
df.filter(df.fname.startswith("T")).show()
df.filter(df.fname.endswith("Cruise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [0]:
#eqNullSafe
from pyspark.sql import Row
df1 = spark.createDataFrame([Row(id=1,value='raj'),
                         Row(id=2,value=None)])
df1.show()

+---+-----+
| id|value|
+---+-----+
|  1|  raj|
|  2| null|
+---+-----+



In [0]:
df1.select( df1.value == 'raj',
           df1.value.eqNullSafe('raj'),
           df1.value.eqNullSafe(None),
           df1.value.eqNullSafe(float('NaN'))
).show()

+-------------+---------------+----------------+---------------+
|(value = raj)|(value <=> raj)|(value <=> NULL)|(value <=> NaN)|
+-------------+---------------+----------------+---------------+
|         true|           true|           false|          false|
|         null|          false|            true|          false|
+-------------+---------------+----------------+---------------+



In [0]:
df.show()
df.printSchema()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)



In [0]:
#isNull & isNotNull
df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()
df.filter(df.gender.isNull()).show()

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
+-----+-----+---+------+



In [0]:
#like , rlike
df.select(df.fname,df.lname,df.id) \
  .filter(df.fname.like("%es")).show()

+-----+-----+---+
|fname|lname| id|
+-----+-----+---+
|James| Bond|100|
+-----+-----+---+



In [0]:
#substring
df.select(df.fname.substr(1,3).alias("substr")).show()


+------+
|substr|
+------+
|   Jam|
|   Ann|
|   Tom|
|   Tom|
+------+



In [0]:
#when & otherwise
from pyspark.sql.functions import when
df.select(df.fname,df.lname,when(df.gender=="M","Male") \
              .when(df.gender=="F","Female") \
              .when(df.gender==None ,"") \
              .otherwise(df.gender).alias("new_gender") \
    ).show()

+----------+-----+----------+
|     fname|lname|new_gender|
+----------+-----+----------+
|     James| Bond|      null|
|       Ann|Varsa|    Female|
|Tom Cruise|  XXX|          |
| Tom Brand| null|      Male|
+----------+-----+----------+



In [0]:
#isin
li=["100","200"]
df.select(df.fname,df.lname,df.id) \
  .filter(df.id.isin(li)) \
  .show()

+-----+-----+---+
|fname|lname| id|
+-----+-----+---+
|James| Bond|100|
|  Ann|Varsa|200|
+-----+-----+---+



In [0]:
#Create DataFrame with struct, array & map
from pyspark.sql.types import StructType,StructField,StringType,ArrayType,MapType
data=[(("James","Bond"),["Java","C#"],{'hair':'black','eye':'brown'}),
      (("Ann","Varsa"),[".NET","Python"],{'hair':'brown','eye':'black'}),
      (("Tom Cruise",""),["Python","Scala"],{'hair':'red','eye':'grey'}),
      (("Tom Brand",None),["Perl","Ruby"],{'hair':'black','eye':'blue'})]

schema = StructType([
        StructField('name', StructType([
            StructField('fname', StringType(), True),
            StructField('lname', StringType(), True)])),
        StructField('languages', ArrayType(StringType()),True),
        StructField('properties', MapType(StringType(),StringType()),True)
     ])
df=spark.createDataFrame(data,schema)
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [0]:
#getField()
# in case of struct field, give sub column name, in case of map type , need to provide a specific key name
df.select(col('name').getField('fname'),df.properties.getField('hair')).show()

+----------+----------------+
|name.fname|properties[hair]|
+----------+----------------+
|     James|           black|
|       Ann|           brown|
|Tom Cruise|             red|
| Tom Brand|           black|
+----------+----------------+



In [0]:
#getItem() used with ArrayType
df.select(df.languages.getItem(1)).show()

#getItem() used with MapType
df.select(df.properties.getItem("hair")).show()

+------------+
|languages[1]|
+------------+
|          C#|
|      Python|
|       Scala|
|        Ruby|
+------------+

+----------------+
|properties[hair]|
+----------------+
|           black|
|           brown|
|             red|
|           black|
+----------------+

